<a href="https://colab.research.google.com/github/vinayshanbhag/python_concepts/blob/master/Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python Decorators

Python decorators can be used to dynamically alter the functionality of a function without having to change the source code of the function being decorated.

The ```decorator``` function takes the original function to be decorated as input. The inner ```decorated``` wraps additional functionality around the original function. The ```decorator``` then returns the ```decorated``` inner function 



In [0]:
def decorator(func_to_be_decorated):
  print(f"decorator called for {func_to_be_decorated.__name__}")
  def decorated():
    print(f"calling {func_to_be_decorated.__name__}:")
    val = func_to_be_decorated()
    print(f":{func_to_be_decorated.__name__} ended")
    return val
  return decorated

To put this to use, we define a function ```baz``` and then reassign ```baz``` to decorated version

In [7]:
def baz():
  print("inside baz")
  return "baz"
baz = decorator(baz)

decorator called for baz


Calling ```baz```, we can see the decorated behavior

In [11]:
x = baz()
print(f"function baz returned: {x}")

calling baz:
inside baz
:baz ended
function baz returned: baz


Alternatively, this can be simplified with the @decorator syntax.

In [9]:
@decorator
def foo():
  print("inside foo")
  return "foo"

@decorator
def bar():
  print("inside bar")
  return "bar"  

decorator called for foo
decorator called for bar


Calling these functions, reveals decorated behavior

In [43]:
v = foo()
print(f"function foo returned: {v}\n")
w = bar()
print(f"function bar returned: {w}\n")

calling foo:
inside foo
:foo ended
function foo returned: foo

calling bar:
inside bar
:bar ended
function bar returned: bar



To handle function arguments, pass ```*args``` and ```**kwargs``` to the wrapper function

In [0]:
def yet_another_decorator(func):
  def decorated(*args, **kwargs):
    print(f"calling {func.__name__}:")
    val = func(*args, **kwargs)
    print(f":{func.__name__} ended")
    return val
  return decorated

In [0]:
@yet_another_decorator
def hello(*args):
  print(f"hello:{' '.join(list(args))}")
  return ' '.join(list(args))

In [46]:
hello('beautiful', 'world')

calling hello:
hello:beautiful world
:hello ended


'beautiful world'

Decorators using classes

In [0]:
class Decorator_class():
  def __init__(self, func_to_be_decorated):
    self.func_to_be_decorated = func_to_be_decorated
  
  def __call__(self, *args, **kwargs):
    print(f"calling {self.func_to_be_decorated.__name__}:")
    val = self.func_to_be_decorated(*args, **kwargs)
    print(f":{self.func_to_be_decorated.__name__} ended")
    return val

In [0]:
@Decorator_class
def some_func(*args):
  print(f"some_func({','.join(list(args))}):")
  return ' '.join(list(args))

In [49]:
z = some_func("test")
print(f"some_func returned:{z}")

calling some_func:
some_func(test):
:some_func ended
some_func returned:test


Chaining multiple decorators

Here we apply ```time_it``` decorator to ```some_long_running_func``` and then also apply the ```logger``` decorator. The problem here is that the inner most decorator returns a wrapper func, so the logger sees ```wrapper``` instead of our original function


In [0]:
import time
def logger(f):
  def wrapper(*args, **kwargs):
    print(f"log: {f.__name__}({','.join([str(i) for i in list(args)])},{','.join([f'{i[0]}={str(i[1])}' for i in kwargs.items()])}):")
    return f(*args, **kwargs)
  return wrapper

def time_it(f):
  def wrapper(*args, **kwargs):
    s = time.perf_counter()
    v = f(*args, **kwargs)
    e = time.perf_counter()
    print(f"{f.__name__} took:{e-s:0.3f}s")
    return v
  return wrapper

In [0]:
@logger
@time_it
def some_long_running_func():
  time.sleep(2)

In [151]:
some_long_running_func()

log: wrapper(,):
some_long_running_func took:2.002s


This is fixed by using the wraps decorator from functools. wrapper function in each decorator is now decorated with ```@wraps(<original func>)```

In [0]:
import time
from functools import wraps
def logger2(f):
  @wraps(f)
  def wrapper(*args, **kwargs):
    print(f"log: {f.__name__}({','.join([str(i) for i in list(args)])},{','.join([f'{i[0]}={str(i[1])}' for i in kwargs.items()])}):")
    return f(*args, **kwargs)
  return wrapper

def time_it2(f):
  @wraps(f)
  def wrapper(*args, **kwargs):
    s = time.perf_counter()
    v = f(*args, **kwargs)
    e = time.perf_counter()
    print(f"{f.__name__} took:{e-s:0.3f}s")
    return v
  return wrapper

In [0]:
@logger2
@time_it2
def yet_another_long_running_func():
  time.sleep(2)

In [154]:
yet_another_long_running_func()

log: yet_another_long_running_func(,):
yet_another_long_running_func took:2.002s
